In [ ]:
!pip install rake_nltk
!pip install konlpy
!pip install krwordrank
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!pip install soynlp==0.0.4

In [229]:
import pandas as pd
from rake_nltk import Rake
from konlpy.tag import Kkma
from konlpy.utils import pprint
from konlpy.tag import Okt  
from collections import Counter
from math import log
from nltk.corpus import stopwords
from krwordrank.word import KRWordRank
from krwordrank.word import summarize_with_keywords
from krwordrank.sentence import summarize_with_sentences
from numpy import dot
from numpy.linalg import norm
import numpy as np
from nltk.tokenize import word_tokenize  
r = Rake()
kkma = Kkma()

In [325]:
text = ['무비 블록을 예상합시다', '개싫다 무비 블록','무비 블록을 통해 영화를 보았다', '예시 문장 입니다','예시 영화를 봅시다','영화는 즐거웠다',' 다시보고 싶다', '영화란 신기한 것이다',' 다른 사람에게 이 영화를 추천해야한다','영화 보기 싫다','영화 개노잼이다','왜 이 영화를 봐야 하는가','영화를 보고싶지 않다','아이디어가 떨어진다','고양이는 귀엽다','그래야만 할것이다','영화를 왜 봐야하는가','난 왜 이걸 하고있는가','영화를 봐야 한다','망할 무비 블록','아 말안듣네','무비 블록을 통해 영화를 보았다']



In [326]:
# 문장 간소화
out_list = []
for token in text:
    remove_list = []
    token_list = word_tokenize(token)
    for remove in token_list:
        remove1 = remove.replace("을", "")
        remove2 = remove1.replace("를","")
        remove_list.append(remove2)
    str_list = " ".join(remove_list)
    out_list.append(str_list)
    
print(out_list)

['무비 블록 예상합시다', '개싫다 무비 블록', '무비 블록 통해 영화 보았다', '예시 문장 입니다', '예시 영화 봅시다', '영화는 즐거웠다', '다시보고 싶다', '영화란 신기한 것이다', '다른 사람에게 이 영화 추천해야한다', '영화 보기 싫다', '영화 개노잼이다', '왜 이 영화 봐야 하는가', '영화 보고싶지 않다', '아이디어가 떨어진다', '고양이는 귀엽다', '그래야만 할것이다', '영화 왜 봐야하는가', '난 왜 이걸 하고있는가', '영화 봐야 한다', '망할 무비 블록', '아 말안듣네', '무비 블록 통해 영화 보았다']


In [331]:
# 키워드 추출
penalty = lambda x:0 if (25 <= len(x) <= 80) else 1
stopwords = {'관람객', '너무', '정말', '진짜'}

keywords, sents = summarize_with_sentences(
    out_list,
    penalty=penalty,
    stopwords = stopwords,
    diversity=0.5,
    num_keywords=10,
    num_keysents=10,
    verbose=False
)

print(keywords)
keywords_list = []
for i in keywords:
    keywords_list.append(i)
print(keywords_list)

# tf-idf vectorizer
def tf(term, document):
    return document.count(term)

def idf(term):
    df = 0
    for doc in text:
        if term in doc:
            df = df + 1
    return log(len(text)/(df+1))+1

def tfidf(term, document):
    return tf(term, document)*idf(term)

tfidf_list = []

for doc in text:
    tfidf_list.append([])
    for token in keywords_list:
        tfidf_list[-1].append(tfidf(token, doc))

tfidf_pd = pd.DataFrame(tfidf_list, columns=keywords_list)
print(tfidf_pd)

#cos similarity
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))
cos_sim_list = []
print('코사인 유사도:', cos_sim(tfidf_pd.values[0], tfidf_pd.values[21]))
for i in range(len(tfidf_list)):
    for j in range(len(tfidf_list)):
        cos_sim_list.append(cos_sim(tfidf_pd.values[i], tfidf_pd.values[j]))
        
print(max(cos_sim_list))
print(cos_sim_list.index(max(cos_sim_list)))
print(cos_sim_list[34])

{'무비': 1.240498173970166, '블록': 0.815498173970166, '영화': 0.6556360204699151}
['무비', '블록', '영화']
          무비        블록        영화
0   2.299283  2.299283  0.000000
1   2.299283  2.299283  0.000000
2   2.299283  2.299283  1.526093
3   0.000000  0.000000  0.000000
4   0.000000  0.000000  1.526093
5   0.000000  0.000000  1.526093
6   0.000000  0.000000  0.000000
7   0.000000  0.000000  1.526093
8   0.000000  0.000000  1.526093
9   0.000000  0.000000  1.526093
10  0.000000  0.000000  1.526093
11  0.000000  0.000000  1.526093
12  0.000000  0.000000  1.526093
13  0.000000  0.000000  0.000000
14  0.000000  0.000000  0.000000
15  0.000000  0.000000  0.000000
16  0.000000  0.000000  1.526093
17  0.000000  0.000000  0.000000
18  0.000000  0.000000  1.526093
19  2.299283  2.299283  0.000000
20  0.000000  0.000000  0.000000
21  2.299283  2.299283  1.526093
코사인 유사도: 0.9052588259701114
1.0000000000000002
46
0.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in double_scalars
